# 一言
とりあえず全てのモデルの予測足して平均した．
## Public Score
# 0.766

In [1]:
#load packages
import sklearn  # collection of machine learning algorithms
from IPython import display  # pretty printing of dataframes in Jupyter notebook
# collection of functions for scientific computing and advance mathematics
import scipy as sp
import numpy as np  # foundational package for scientific computing
# collection of functions for scientific and publication-ready visualization
import matplotlib
import pandas as pd  # collection of functions for data processing and analysis modeled after R dataframes with SQL like features
from xgboost import XGBClassifier
from sklearn import metrics
from sklearn import model_selection
from sklearn import feature_selection
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from pandas_profiling import ProfileReport
import sweetviz as sv
import scipy as sp
import string
import re as re
import io
import zipfile
import requests
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
import warnings
import time
import random
import IPython
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from pandas import Series, DataFrame
import sys  # access to system parameters https://docs.python.org/3/library/sys.html
print("Python version: {}". format(sys.version))

print("pandas version: {}". format(pd.__version__))

print("matplotlib version: {}". format(matplotlib.__version__))

print("NumPy version: {}". format(np.__version__))

print("SciPy version: {}". format(sp.__version__))

print("IPython version: {}". format(IPython.__version__))

print("scikit-learn version: {}". format(sklearn.__version__))

#misc libraries


#ignore warnings
warnings.filterwarnings('ignore')
print('-'*25)
# データインポートライブラリ

# データ加工・処理・分析ライブラリ


# 可視化ライブラリ
%matplotlib inline


#Model
%precision 3


SEED = 42


Python version: 3.9.2 (tags/v3.9.2:1a79785, Feb 19 2021, 13:44:55) [MSC v.1928 64 bit (AMD64)]
pandas version: 1.3.2
matplotlib version: 3.4.3
NumPy version: 1.22.4
SciPy version: 1.7.1
IPython version: 7.26.0
scikit-learn version: 0.24.2
-------------------------


In [2]:
def concat_df(train_data, test_data, sort=True, drop=True):
    """
    overview
    >return a  concatenated df of training and test data
    needed
    >pandas as pd
    input
    >train_data{DataFrame}:train_data
    >test_data{DataFrame}:test_data
    >sort{bool}:True or False
    """
    return pd.concat([train_data, test_data], sort=sort).reset_index(drop=drop)


def divide_df(all_data, target, len=890):
    """
    overview
    > Returns divided dfs of training and test set
    needed
    >
    input
    > all_data{DataFrame}:Train+Test
    > target{str} :Target columns
    > len(int):len of train_data
    
    """
    return all_data.loc[:len], all_data.loc[len:].drop([target], axis=1)


In [3]:
df_train = pd.read_csv("./df/EDA_05_train.csv")
df_test = pd.read_csv("./df/EDA_05_test.csv")
Target = 'Survived'
Final_target = 'Perished'
df_test.shape


(418, 44)

In [4]:
df_all = concat_df(df_train, df_test)
drop_cols = ['Deck', 'Embarked', 'Family', 'Family_Size', 'Family_Size_Grouped', Target,
             'Name', 'Parch', 'PassengerId', 'Pclass', 'Sex', 'SibSp', 'Ticket', 'Title',
             'Ticket_Survival_Rate', 'Family_Survival_Rate', 'Ticket_Survival_Rate_NA', 'Family_Survival_Rate_NA']

df_all.drop(columns=drop_cols, inplace=True)

df_all.head()


,Age,Deck_1,Deck_2,Deck_3,Deck_4,Embarked_1,Embarked_2,Embarked_3,Family_Size_Grouped_1,Family_Size_Grouped_2,...,Pclass_3,Sex_1,Sex_2,Survival_Rate,Survival_Rate_NA,Ticket_Frequency,Title_1,Title_2,Title_3,Title_4
0,2,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,1.0,0.383838,0.0,1,0.0,0.0,0.0,1.0
1,7,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.000000,1.0,2,0.0,0.0,1.0,0.0
2,4,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,...,1.0,1.0,0.0,0.383838,0.0,1,0.0,0.0,1.0,0.0
3,7,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.383838,0.0,2,0.0,0.0,1.0,0.0
4,7,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,...,1.0,0.0,1.0,0.383838,0.0,1,0.0,0.0,0.0,1.0


In [5]:
X_train = StandardScaler().fit_transform(df_train.drop(columns=drop_cols))
y_train = df_train[Target].values
X_test = StandardScaler().fit_transform(df_test.drop(columns=drop_cols))
print('X_train shape: {}'.format(X_train.shape))
print('y_train shape: {}'.format(y_train.shape))
print('X_test shape: {}'.format(X_test.shape))


X_train shape: (891, 26)
y_train shape: (891,)
X_test shape: (418, 26)


In [6]:
#Machine Learning Algorithm (MLA) Selection and Initialization
MLA = [
    #Ensemble Methods
    ensemble.AdaBoostClassifier(),
    ensemble.BaggingClassifier(),
    ensemble.ExtraTreesClassifier(),
    ensemble.GradientBoostingClassifier(),
    ensemble.RandomForestClassifier(),

    #Gaussian Processes
    gaussian_process.GaussianProcessClassifier(),

    #GLM
    linear_model.LogisticRegressionCV(),
    linear_model.SGDClassifier(),
    linear_model.Perceptron(),

    #Navies Bayes
    naive_bayes.BernoulliNB(),
    naive_bayes.GaussianNB(),

    #Nearest Neighbor
    neighbors.KNeighborsClassifier(),

    #SVM
    svm.SVC(probability=True),
    svm.NuSVC(probability=True),
    svm.LinearSVC(),

    #Trees
    tree.DecisionTreeClassifier(),
    tree.ExtraTreeClassifier(),

    #Discriminant Analysis
    discriminant_analysis.LinearDiscriminantAnalysis(),
    discriminant_analysis.QuadraticDiscriminantAnalysis(),


    #xgboost: http://xgboost.readthedocs.io/en/latest/model.html
    XGBClassifier()
]


#split dataset in cross-validation with this splitter class: http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.ShuffleSplit.html#sklearn.model_selection.ShuffleSplit
#note: this is an alternative to train_test_split
# run model 10x with 60/30 split intentionally leaving out 10%
cv_split = model_selection.ShuffleSplit(
    n_splits=10, test_size=.3, train_size=.6, random_state=0)

#create table to compare MLA metrics
MLA_columns = ['MLA Name', 'MLA Parameters', 'MLA Train Accuracy Mean',
               'MLA Test Accuracy Mean', 'MLA Test Accuracy 3*STD', 'MLA Time']
MLA_compare = pd.DataFrame(columns=MLA_columns)

#create table to compare MLA predictions
MLA_predict = y_train
model_list = {}
score_list =[]
#index through MLA and save performance to table
row_index = 0
for alg in MLA:

    #set name and parameters
    MLA_name = alg.__class__.__name__
    MLA_compare.loc[row_index, 'MLA Name'] = MLA_name
    MLA_compare.loc[row_index, 'MLA Parameters'] = str(alg.get_params())

    #score model with cross validation: http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html#sklearn.model_selection.cross_validate
    cv_results = model_selection.cross_validate(
        alg, X_train, y_train, cv=cv_split, return_train_score=True,return_estimator=True)

    MLA_compare.loc[row_index, 'MLA Time'] = cv_results['fit_time'].mean()
    MLA_compare.loc[row_index,
                    'MLA Train Accuracy Mean'] = cv_results['train_score'].mean()
    MLA_compare.loc[row_index,
                    'MLA Test Accuracy Mean'] = cv_results['test_score'].mean()
    #if this is a non-bias random sample, then +/-3 standard deviations (std) from the mean, should statistically capture 99.7% of the subsets
    # let's know the worst that can happen!
    MLA_compare.loc[row_index,
                    'MLA Test Accuracy 3*STD'] = cv_results['test_score'].std()*3
        
    #save MLA predictions - see section 6 for usage
    alg.fit(X_train, y_train)
    
    score_list.append(str(round(cv_results['train_score'].mean(),4)))
    try:
        model_list[str(round(cv_results['train_score'].mean(),4))]=alg.predict_proba(X_test)
    except:
        pass

    row_index += 1

score_list.sort(reverse=True)
#print and sort table: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sort_values.html
MLA_compare.sort_values(by=['MLA Test Accuracy Mean'],
                        ascending=False, inplace=True)
MLA_compare


,MLA Name,MLA Parameters,MLA Train Accuracy Mean,MLA Test Accuracy Mean,MLA Test Accuracy 3*STD,MLA Time
17,LinearDiscriminantAnalysis,"{'covariance_estimator': None, 'n_components':...",0.855993,0.853358,0.047241,0.005063
3,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...",0.915543,0.851866,0.061322,0.09723
6,LogisticRegressionCV,"{'Cs': 10, 'class_weight': None, 'cv': None, '...",0.861049,0.850373,0.066121,0.460674
14,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...",0.857303,0.848134,0.058392,0.037232
0,AdaBoostClassifier,"{'algorithm': 'SAMME.R', 'base_estimator': Non...",0.865356,0.843284,0.065272,0.082403
12,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...",0.880524,0.842537,0.058338,0.071007
19,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...",0.954494,0.840672,0.039434,0.101232
5,GaussianProcessClassifier,"{'copy_X_train': True, 'kernel': None, 'max_it...",0.916854,0.838806,0.055475,0.219156
13,NuSVC,"{'break_ties': False, 'cache_size': 200, 'clas...",0.854494,0.837313,0.045388,0.081081
4,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",0.96161,0.834328,0.049101,0.257528


In [7]:
print(score_list)
pick = score_list[:10]
pick

['0.9616', '0.9616', '0.9616', '0.9616', '0.9545', '0.9507', '0.9169', '0.9155', '0.8805', '0.8654', '0.8652', '0.861', '0.8573', '0.856', '0.8545', '0.8178', '0.8004', '0.7766', '0.7625', '0.7603']


['0.9616',
 '0.9616',
 '0.9616',
 '0.9616',
 '0.9545',
 '0.9507',
 '0.9169',
 '0.9155',
 '0.8805',
 '0.8654']

In [51]:

for i in range(0,len(pick)-1):
    if i == 0:
        pred = model_list[str(score_list[i])]
    else:
        try:
            pred += model_list[str(score_list[i])]
        except:
            pass

predict_prob = (pred/len(model_list))
predict= predict_prob.argmax(axis=1)
predict


array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0,

In [52]:
submission_ex = pd.read_csv("gender_submission.csv")
submission_df = pd.DataFrame(columns=['PassengerId', Target])
submission_df['PassengerId'] = df_test['PassengerId']
submission_df[Target] = predict
submission_df[Final_target] = 0
submission_df.loc[submission_df[Target] == 0, Final_target] = 1
submission_df.loc[submission_df[Target] == 1, Final_target] = 0
submission_df.drop(Target, axis=1, inplace=True)

submission_df.reset_index(drop=True, inplace=True)
submission_df.to_csv('sub/07_submissions.csv', header=True, index=False)
print("Submission Format:{} ".format(submission_ex.shape))
print("My Sumbisison Format:{}".format(submission_df.shape))
submission_df

Submission Format:(418, 2) 
My Sumbisison Format:(418, 2)


,PassengerId,Perished
0,892,1
1,893,1
2,894,1
3,895,1
4,896,0
...,...,...
413,1305,1
414,1306,0
415,1307,1
416,1308,1


In [53]:
submission_ex = pd.read_csv("gender_submission.csv")
submission_df = pd.DataFrame(columns=['PassengerId', Target])
submission_df['PassengerId'] = df_test['PassengerId']
submission_df.reset_index(drop=True, inplace=True)
submission_df[Target] = predict
submission_df.to_csv('sub/07_submissions_kaggle.csv', header=True, index=False)
print("Submission Format:{} ".format(submission_ex.shape))
print("My Sumbisison Format:{}".format(submission_df.shape))
submission_df

Submission Format:(418, 2) 
My Sumbisison Format:(418, 2)


,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
